In [1]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 36835622912 (34.31 GB)
DFS Remaining: 36835573760 (34.31 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.20.0.4:9866 (cs544_tester_directory-dn-1.cs544_tester_directory_default)
Hostname: 4a6732874b8a
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 7386464256 (6.88 GB)
DFS Remaining: 18417786880 (17.15 GB)
DFS Used%: 0.00%
DFS Remaining%: 71.

In [2]:
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2023-10-27 01:06:00--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv.1’

hdma-wi-2021.csv.1  100%[===================>] 166.84M  82.4MB/s    in 2.0s    

2023-10-27 01:06:02 (82.4 MB/s) - ‘hdma-wi-2021.csv.1’ saved [174944099/174944099]



In [3]:
!ls

hdma-wi-2021.csv  hdma-wi-2021.csv.1  p4.ipynb	p4a.ipynb  p4b.ipynb


In [4]:
!rm  hdma-wi-2021.csv.1

In [5]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [6]:
# 1 megabyte (MB) = 1,048,576 bytes, source https://kb.iu.edu/d/ackw#:~:text=1%20MB%20is%201%2C024%20kilobytes,1%2C610%20CDs%20worth%20of%20data.
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv

In [7]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [8]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [9]:
import requests

In [10]:
#q3
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.raise_for_status() # make an exception is not 200 (not success)
r.json()

{'FileStatus': {'accessTime': 1698368772560,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698368778707,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [11]:
#q4
# source: https://hadoop.apache.org/docs/r1.0.4/webhdfs.html #OPEN
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.raise_for_status() # make an exception is not 200 (not success)
r.json()["Location"]

'http://04f80de43ac5:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [12]:
# !hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
# !hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [13]:
#q5
import requests
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN"
container_number_count = dict()


for offset in range(0, 174944099, 1048576):
    url_with_offset = f"{url}&offset={offset}"
    r = requests.get(url_with_offset,stream = True)

    if r.status_code == 200:
        url_array = r.url.split('/')
        # print(url_array)
        container_id = url_array[2]
        container = container_id.split(':')
        final_container_id = container[0]
        if final_container_id not in container_number_count:
            container_number_count[final_container_id] = 1
        else:
            container_number_count[final_container_id] += 1

print(dict(container_number_count))

{'04f80de43ac5': 92, '4a6732874b8a': 75}


In [14]:
#q6
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

with hdfs.open_input_file("hdfs://boss:9000/single.csv") as f:
    first_10 = f.read(10)
    print(first_10)
    

2023-10-27 01:06:50,726 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'


In [15]:
#q7
import io
hdfs = pa.fs.HadoopFileSystem("boss",9000)

with hdfs.open_input_file("hdfs://boss:9000/single.csv") as f:
    count = 0
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for line in reader:
        if "Single Family" in line:
            count += 1

print(count)

444874
